In [1]:
from enum import Enum
from typing import Literal
from pydantic import BaseModel

class AgeGroup(str, Enum):
    CHILD = 'child'
    TEEN = 'teen'
    ADULT = 'adult'
    SENIOR = 'senior'

class Person(BaseModel):
    name: str
    age_group: str


In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType
from camel.agents import ChatAgent

model = ModelFactory.create(model_platform=ModelPlatformType.OLLAMA, model_type="qwen3:14b", model_config_dict= {"temperature": 0.4, "stream": True})
agent = ChatAgent(model= model, system_message="You are a helpful assistant", token_limit=4096)

Ollama server started on http://localhost:11434/v1 for qwen3:14b model.


In [ ]:
res = agent.step("What is 1+1?")

In [ ]:
print(res.msgs[0].content)

<think>
Okay, the user asked "What is 1+1?" That's a basic math question. I need to make sure I answer correctly. Let me think... 1 plus 1 equals 2. That's straightforward. But maybe they want more than just the number? Maybe they're testing if I know simple arithmetic. I should confirm the answer and maybe add a friendly note. Let me check if there's any context I'm missing. The user didn't specify any special conditions, so it's safe to assume standard addition. Alright, the answer is 2. I'll present that clearly and offer further help in case they have more questions.
</think>

1 + 1 equals **2**. 

Let me know if you need help with anything else! 😊


In [ ]:
import yaml
import json
from jinja2 import Environment, FileSystemLoader, Template

with open("src/prompts/client.yaml") as f:
    prompts = yaml.safe_load(f)

template = Template(prompts["profile"])
data = json.loads(open("data/characters/clients.json").read())[0]
print(
    template.render(
        data=data,
        therapist={"name": "Dr. Smith", "expertise": "Cognitive Behavioral Therapy"},
        previous_sessions=[
            {
                "id": 1,
                "date": "2023-10-01",
                "summary": "Discussed anxiety management techniques.",
            },
            {
                "id": 2,
                "date": "2023-10-01",
                "summary": "Discussed anxiety management techniques.",
            },
        ],
    )
)

From now on, imagine you are John Doe, a 25 year-old male. 
You are currently Employed and have Moderate income. 
You are currently Single, and have 0 friends in your social circle.
You are generally described as Introverted, analytical, and detail-oriented. Prefers to work alone and often feels overwhelmed in social situations. Highly conscientious and organized, but struggles with anxiety and self-doubt.

For the past 6 months, you feel a lot of pressure to perform well in your research, and often feel overwhelmed by the expectations placed on you.. 
In particular, the following things have been bothering you: Deadlines, Peer pressure, High expectations.
You have been the following to cope with these issues: Meditation, Procrastination, Working out, Smoking.

Recently, you decided to seek help from a therapist, and met Dr. Smith, a therapist who specializes in Cognitive Behavioral Therapy. This is your 3rd  session together.
Here's a summary of your previous sessions:
1: Discussed an

In [ ]:
from langchain.chat_models import init_chat_model, ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

model_name = "qwen3:14b"
# client = init_chat_model(model=model_name, model_provider="ollama", temperature=0.6)
client = init_chat_model(model=model_name, model_provider="openai", base_url="http://localhost:11434/v1", api_key="ollama")
# client = ChatOpenAI(
#         model=model_name,
#         base_url="http://localhost:11434/v1",
#         api_key="http://localhost:11434/v1",
# )
model = client.with_structured_output(Joke)

res = model.invoke("tell me a joke")

In [ ]:
from typing import Dict, List, Any
def set_therapist(prev_sessions: List[Dict[str, str] | None] = []):
    print(prev_sessions)
set_therapist([{"summary": "Discussed anxiety management techniques."}, None, {"summary": "Discussed anxiety management techniques."}])

[{'summary': 'Discussed anxiety management techniques.'}, None, {'summary': 'Discussed anxiety management techniques.'}]


In [ ]:
import json
from pydantic import BaseModel, Field

class MentalState(BaseModel):
    Emotion: str = Field(
        description="Current emotion based on Ekman's 8 emotions (label only)",
        default="Unknown",
    )
    Beliefs: str = Field(
        description="Current beliefs (1-2 sentences)", default="Unknown"
    )
    Desires: str = Field(
        description="Current desires (1-2 sentences)", default="Unknown"
    )
    Intents: str = Field(
        description="Current intentions (1-2 sentences) ", default="Unknown"
    )
    Trust_Level: int = Field(
        description="Crrent level of trust towards the therapist (0-100)",
        default=0,
    )

mental_state = MentalState(Emotion= "Happy", 
                           Beliefs="I believe I can improve with therapy.",
                           Desires="I desire to feel more relaxed and less anxious.",
                           Intents="I intend to practice mindfulness techniques daily.",
                           Trust_Level=80)


mental_state.model_dump()["Emotion"]

'Happy'

In [ ]:
from langchain.chat_models import init_chat_model
AZURE_API_KEY="7abfe5b315a145e49287fa3a0cdeb4d9"
OPENAI_API_KEY="sk-4d3b1c7f8e2a4b0f9c5d6e7f8a9b0c1d"
client = init_chat_model(
    model="gpt-4",  # or your specific model name
    model_provider="openai",
    api_key=OPENAI_API_KEY,
)

In [ ]:
client.invoke(
    "What is the capital of France?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-4d3b1***********************0c1d. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}